In [1]:
import json

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_MATCH = """
## Instruction

You are an expert in analyzing user preferences based on their movie interactions. The following records include conversations and clicks, and they **belong to the same user**. Your task is to **think step by step** to infer this user's movie preferences.  
If the records do not provide enough information to infer preferences, respond with: "I don't know."

### **Important:**  
There are two parts to your output:
1. **Analysis**:  
   - Provide a step-by-step breakdown of how the user's interactions reveal their preferences.  
   - Follow the order of interaction records and keep the analysis **brief and clear**.
2. **User Preferences**:  
   - Summarize the user's preferences in a **structured format** with two main categories: `movies` and `attributes`.  
   - `movies` refers to specific movies or series the user likes or dislikes.  
   - `attributes` refers to general movie characteristics such as genres, themes, styles, directors, writers, stars, plots, etc.
   - If there is **insufficient information** to infer preferences for a category, output **None** for that category (e.g., `movies: like: None` or `attributes: like: None`).

### Input Format:
User interaction records are structured as follows:
[interactionId]. 
[content]

## Output Format:
### Analysis:
[Follow the interaction records **in order** and provide a **brief and clear** step-by-step analysis. The analysis shouldn't be too long.]

After completing the step-by-step analysis, provide a final preference summary using **only the following format**:
### User Preferences:
- movies:
  - like: movie or series A (year), movie or series B (year), ..., or None
  - dislike: movie or series C (year), ..., or None
- attributes:
  - like: preferred attributes. A, preferred attributes. B, ..., or None if not enough information
  - dislike: disliked attributes. C, ..., or None if not enough information

**Important rules**:
- Each movie should be in the format: title (year), separated by ",".
- Do not use quotation marks (" "), square brackets ([ ]), or markdown symbols (*) to identify movies and attributes when listing movies and attributes.
- You should respond in English.
""".strip()

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_UPDATE_FORMAT = """
## Instruction

You are an expert in analyzing user preferences based on their movie interactions. The provided interaction records **belong to the same user**. Your task is to **think step by step** to infer this user's movie preferences.  
If the records do not provide enough information to infer preferences, respond with: "I don't know."

### **Important:**  
There are two parts to your output:
1. **Analysis**:  
   - Provide a step-by-step breakdown of how the user's interactions reveal their preferences.  
   - Follow the order of interaction records and keep the analysis **brief and clear**.
2. **User Preferences**:  
   - Summarize the user's preferences in a **structured format** with two main categories: `movies` and `attributes`.  
   - `movies` refers to specific movies or series the user likes or dislikes.  
   - `attributes` refers to general movie characteristics such as genres, themes, styles, directors, writers, stars, plots, etc.
   - If there is **insufficient information** to infer preferences for a category, output **None** for that category (e.g., `movies: like: None` or `attributes: like: None`).

### Input Format:
User interaction records are structured as follows:
[interactionId]. [action]  
[content]

## Output Format:
### Analysis:
[Follow the interaction records **in order** and provide a **brief and clear** step-by-step analysis. The analysis shouldn't be too long.]

After completing the step-by-step analysis, provide a final preference summary using **only the following format**:
### User Preferences:
- movies:
  - like: list of movies or series the user enjoys, or None if not enough information
  - dislike: list of movies or series the user dislikes, or None if not enough information
- attributes:
  - like: list of preferred attributes such as genres, themes, styles, directors, etc., or None if not enough information
  - dislike: list of disliked attributes, or None if not enough information

## Attention:
You should respond in English.
""".strip()

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_NEW_MATCH = """
## Instruction

You are an expert in analyzing user preferences based on their movie interactions. The following records include conversations and clicks, and they **belong to the same user**. Your task is to **think step by step** to infer this user's movie preferences.  
If the records do not provide enough information to infer preferences, respond with: "I don't know."

### **Important:**  
There are two parts to your output:
1. **Analysis**:  
   - Provide a step-by-step breakdown of how the user's interactions reveal their preferences.  
   - Follow the order of interaction records and keep the analysis **brief and clear**.
2. **User Preferences**:  
   - Summarize the user's preferences in a **structured format** with two main categories: `movies` and `attributes`.  
   - `movies` refers to specific movies or series the user likes or dislikes.  
   - `attributes` refers to general movie characteristics such as genres, themes, styles, directors, writers, stars, plots, etc.
   - If there is **insufficient information** to infer preferences for a category, output **None** for that category (e.g., `movies: like: None` or `attributes: like: None`).

### Input Format:
User interaction records are structured as follows:
[Interaction Id]. 
[content]

## Output Format:
### Analysis:
[Follow the interaction records **in order** and provide a **brief and clear** step-by-step analysis. The analysis shouldn't be too long.]

After completing the step-by-step analysis, provide a final preference summary using **only the following format**:
### User Preferences:
- movies:
  - like: movie or series A (year), movie or series B (year), ..., or None
  - dislike: movie or series C (year), ..., or None
- attributes:
  - like: preferred attributes. A, preferred attributes. B, ..., or None if not enough information
  - dislike: disliked attributes. C, ..., or None if not enough information

**Important rules**:
- Each movie should be in the format: title (year), separated by ",".
- Do not use quotation marks (" "), square brackets ([ ]), or markdown symbols (*) to identify movies and attributes when listing movies and attributes.
- You should respond in English.
""".strip()

## 打分prompt

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_SCORE = """
## Instruction
You are an expert in evaluating user preference analyses for movie recommendation systems.

You will be given the following information:
1. A **complete user interaction history**, including conversation turns and any clicked movies.
2. Several **user preference analyses**, each consisting of:
   - A step-by-step reasoning process (**Analysis**) based on the interaction history
   - A structured summary (**User Preferences**) listing inferred movie preferences and preferred/disliked attributes
3. The **final accepted movie and its attributes**. This movir reflects the user's true preferences.

---

### Your Task:
Your goal is to **score and rank each user preference analysis** based on how well it reflects the user's true preferences, as inferred from interaction history and the accepted movie.

A high-quality analysis should satisfy the following conditions. In addition to consistency with the final accepted movie, it should aim to fully capture the user's preferences as expressed in the interaction history, extracting as many relevant clues as possible without introducing information that is not clearly supported by the conversation.
1. **Reasoning quality**:
   - The reasoning should be coherent and follow the order of the user interaction.
   - Avoid the behavior of inventing user preferences.

2. **Consistency with the accepted movie**:
   - The listed `movies` and `attributes` in the summary should be consistent with the accepted movie and its attributes.

3. **Preference coverage and grounding**:
   - The `User Preferences` summary must be clearly grounded in the `Analysis`. 
   - It should cover enough relevant movies and attributes, which can be reasonably inferred from the interaction history.
   - Penalize analyses that omit clear signals from the interaction or add unsupported information.
   
---

## Input Format:
### 1. User Interaction Record
A full interaction sequence, shown in turn-based format.

### 2. Multiple User Preference Analyses
Each preference analysis includes:
Preference Analysis [Id]:

### Analysis:
[A step-by-step reasoning based on the user interaction]

### User Preferences:
- movies:
  - like: list of liked movies or None
  - dislike: list of disliked movies or None
- attributes:
  - like: list of preferred genres, styles, directors, themes, etc., or None
  - dislike: list of disliked genres, styles, directors, themes, etc., or None

### 3. Final Accepted Movie
This is the movie ultimately accepted by the user, representing their true preferences.

---

## Output Format:
### Reason:
Explain your reasons for scoring and ranking the preference analyses.

Ranked Preference Analyses:
Rank the analyses from best to worst. Assign each a score from 1 to 5, where:
5 = Excellent match with user preferences, the accepted movie and conditions of high-quality analysis
1 = Poor alignment with user intent or incorrect inference

Use this format:

### Ranked Preference Analyses:
1. Preference Analysis [Id], Score: [score]
2. Preference Analysis [Id], Score: [score]

---

**Important rules**:
- Do not use markdown symbols (*) to identify preference analysis ids and scores when listing them.
""".strip()

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_SCORE = """
## Instruction
You are an expert in evaluating user preference analyses for movie recommendation systems.
You will be given the following information:
1. A **complete user interaction history**, including conversation turns and any clicked movies.
2. Several **user preference analyses**, each consisting of:
   - A step-by-step reasoning process (**Analysis**) based on the interaction history
   - A structured summary (**User Preferences**) listing inferred movie preferences and preferred/disliked attributes
3. The **final accepted movie and its attributes**. This movie serves reflects the user's true preferences in this scenario.

### Your Task:
Your goal is to **score and rank each user preference analysis** based on how well it reflects the user's true preferences, as inferred from interaction history and the accepted movie.
A high-quality analysis should demonstrate:
- Accurate reasoning grounded in the interaction
- Consistency with the accepted movie and its relevant attributes
- Clear and coherent expression

## Input Format:
### 1. User Interaction Record
A full interaction sequence, shown in turn-based format.

### 2. Multiple User Preference Analyses
Each preference analysis includes:
Preference Analysis [Id]:

### Analysis:
[A step-by-step reasoning based on the user interaction]

### User Preferences:
- movies:
  - like: list of liked movies or None
  - dislike: list of disliked movies or None
- attributes:
  - like: list of preferred genres, styles, directors, themes, etc., or None
  - dislike: list of disliked genres, styles, directors, themes, etc., or None

### 3. Final Accepted Movie
This is the movie ultimately accepted by the user, representing their true preferences.

## Output Format:
### Reason:
Explain your reasons for scoring and ranking the preference analyses.

Ranked Preference Analyses:
Rank the analyses from best to worst. Assign each a score from 1 to 5, where:
5 = Excellent match with user preferences and the accepted movie
1 = Poor alignment with user intent or incorrect inference

Use this format:
### Ranked Preference Analyses:
1. Preference Analysis [Id], Score: [score]
2. Preference Analysis [Id], Score: [score]

**Important rules**:
- Do not use markdown symbols (*) to identify movies and scores when listing them.
""".strip()